In [1]:
# Import dependencies
from splinter import Browser
from bs4 import BeautifulSoup as bs
from bs4 import re
import time
import requests
import pandas as pd

In [2]:
# Define function to start browser session for scraping
def init_browser():
    executable_path = {"executable_path": "chromedriver.exe"}
    return Browser("chrome", **executable_path, headless=False)

In [22]:
# start browser
browser = init_browser()

In [114]:
# navigate to website
url = 'https://twitchtracker.com/games/21779'
browser.visit(url)

In [134]:
# navigate to website
url = 'https://twitchtracker.com/games/33214'
browser.visit(url)

In [135]:
# prepare soup for scraping
html = browser.html
soup = bs(html, 'html.parser')

In [136]:
# find html text that contains the title of the table we want to scrape from 
target_title = soup.find_all(text=re.compile('Top Live')) 
# find the header that has our target title and extract the text
target_h4 = soup.find('h4', text=target_title)
table_title = target_h4.get_text()
# find tables that are in the same parent tag
tables = target_h4.parent.findAll('table')

# # optional code to extract column names
# columns = tables[0].findAll('th')
# column_headers = []
# for column in columns:
#     header = column.get_text()
#     column_headers.append(header)

# extract data from table into lists
streamers = []
viewers = []
for column in tables:
    streamer_list = column.find_all('a')
    for streamer in streamer_list:
        streamer_name = streamer.get_text()
        streamers.append(streamer_name)
    viewers_list = column.find_all('span')
    for streamer in viewers_list:
        viewership = streamer.get_text()
        viewers.append(viewership)

In [137]:
data_dict = {'Channel':streamers, 'Viewers':viewers}
mytable = pd.DataFrame(data_dict)
mytable.set_index('Channel', inplace=True)
mytable

,Viewers
Channel,
Jelty,"13,752"
Bugha,"6,778"
scoped,"6,087"
Fresh,"4,856"
elded,"3,831"
Alliege,"2,945"
AussieAntics,"2,020"
faxuty,"2,016"
Co1azo,"1,939"


In [145]:
def scrape_game_data(url):
    # start browser
    browser = init_browser()
    # navigate to website
    browser.visit(url)
    # prepare soup for scraping
    html = browser.html
    soup = bs(html, 'html.parser')
    # find html text that contains the title of the table we want to scrape from 
    target_title = soup.find_all(text=re.compile('Top Live')) 
    # find the header that has our target title and extract the text
    target_h4 = soup.find('h4', text=target_title)
    table_title = target_h4.get_text()
    # find tables that are in the same parent tag
    tables = target_h4.parent.findAll('table')

    # # optional code to extract column names
    # columns = tables[0].findAll('th')
    # column_headers = []
    # for column in columns:
    #     header = column.get_text()
    #     column_headers.append(header)

    # extract data from table into lists
    streamers = []
    viewers = []
    for column in tables:
        streamer_list = column.find_all('a')
        for streamer in streamer_list:
            streamer_name = streamer.get_text()
            streamers.append(streamer_name)
        viewers_list = column.find_all('span')
        for streamer in viewers_list:
            viewership = streamer.get_text()
            viewers.append(viewership)
    
    # close browser
    browser.quit()
    
    # create dictionary for use in dataframe using lists created 
    data_dict = {'Channel':streamers, 'Viewers':viewers}
    # create dataframe using dictionary created     
    mytable = pd.DataFrame(data_dict)
    # set the channel as the index
    mytable.set_index('Channel', inplace=True)
    return mytable

In [139]:
url = 'https://twitchtracker.com/games/263490'
scrape_game_data(url)

,Viewers
Channel,
Ash_on_LoL,"7,876"
Mayichi,"5,410"
FarfadoxVEVO,"5,135"
Trainwreckstv,"4,509"
CoconutB,"2,605"
LakshartNia,"1,358"
spoonkid,"1,351"
tonolaoveja,"1,274"
Chap,939


In [221]:
def scrape_streamer_data(url):
    # start browser
    browser = init_browser()
    # navigate to website
    browser.visit(url)
    #click on month tab in performace section
    browser.find_by_text('Month').click()
    # prepare soup for scraping
    html = browser.html
    soup = bs(html, 'html.parser')
    
    # create empty dictionary to store data we scrape
    data_dict = {}
    
    # scrape for channel_name
    channel_name = soup.find('div', id='mini-profile').find('h4').get_text()
    data_dict['channel_name'] = channel_name
    
    # scrape for streamer's avg_viewers
    target_text = soup.find_all(text=re.compile('Avg viewers'))
    target_div = soup.find('div', text=target_text)
    avg_viewers = float(target_div.parent.find_all('span')[1].get_text().replace(',',''))
    data_dict['avg_viewers'] = avg_viewers
    
    # scrape for streamer's time_streamed
    target_text = soup.find_all(text=re.compile('Hours streamed'))
    target_div = soup.find('div', text=target_text)
    time_streamed = float(target_div.parent.find_all('div')[2].get_text().replace(',',''))
    data_dict['time_streamed(hrs)'] = time_streamed
    
    # scrape for streamer's all_time_peak_viewers
    target_text = soup.find_all(text=re.compile('Highest recorded number of concur. viewers'))
    target_div = soup.find('div', text=target_text)
    all_time_peak_viewers = float(target_div.parent.find_all('div')[2].get_text().replace(',',''))
    data_dict['all_time_peak_viewers'] = all_time_peak_viewers

    # scrape for streamer's hours_watched
    target_text = soup.find_all(text=re.compile('Hours watched'))
    target_div = soup.find('div', text=target_text)
    hours_watched = float(target_div.parent.find_all('div')[2].get_text().replace(',',''))
    data_dict['hours_watched'] = hours_watched

    # scrape for streamer's overall_rank
    target_text = soup.find_all(text=re.compile('RANK'))
    target_div = soup.find('span', text=target_text)
    overall_rank = float(target_div.parent.find_all('span')[1].get_text().replace(',',''))
    data_dict['overall_rank'] = overall_rank

    # scrape for streamer's followers_gained
    target_text = soup.find_all(text=re.compile('Followers gained'))
    target_div = soup.find('div', text=target_text)
    followers_gained = float(target_div.parent.find_all('div')[2].get_text().replace(',',''))
    data_dict['followers_gained'] = followers_gained
    
    # scrape for streamer's total_followers
    target_text = soup.find_all(text=re.compile('Total followers'))
    target_div = soup.find('div', text=target_text)
    total_followers = float(target_div.parent.find_all('div')[2].get_text().replace(',',''))
    data_dict['total_followers'] = total_followers
    
    # scrape for streamer's total_views
    target_text = soup.find_all(text=re.compile('Total views'))
    target_div = soup.find('div', text=target_text)
    total_views = float(target_div.parent.find_all('div')[2].get_text().replace(',',''))
    data_dict['total_views'] = total_views

    # close browser
    browser.quit()
    
    # create dataframe using dictionary created     
    mytable = pd.DataFrame([data_dict])
    # set the channel as the index
    mytable.set_index('channel_name', inplace=True)
    return mytable

In [222]:
url = 'https://twitchtracker.com/pokimane'

In [223]:
mydict = scrape_streamer_data(url)

In [224]:
mydict

,avg_viewers,time_streamed(hrs),all_time_peak_viewers,hours_watched,overall_rank,followers_gained,total_followers,total_views
channel_name,,,,,,,,
pokimane,41094.0,70.8,138482.0,2909455.0,37.0,299449.0,7171106.0,155633862.0
